**Zadanie 5. (4p)** Napisz program, który **sprawdza**, jaka jest wielkość list postingowych dla słów
z Wikipedyjki, gdy:

1. Kodujemy numery dokumentów za pomocą `int32`.
2. Kodujemy różnice numerów dokumentów za pomocą kodu VB.
3. Kodujemy różnice numerów dokumentów za pomocą kodu VB, operującego 4-bitowymi (a nie 8)- porcjami danych.
4. Kodujemy różnice numerów dokumentów za pomocą kodu Gamma (wybierz najbardziej wydajny
wariant, biorąc pod uwagę że nie musisz kodować liczby 0). Zaokrąglaj wielkości całych list
postingowych do liczby bitów podzielnej przez 8.

Ważne: wynikiem programu mają być 4 liczby, nie musisz zatem implementować tych kodów. Utożsamiaj małe i duże literki, ale możesz się nie przejmować tokenizacją (wystarczy split, choć możesz
też zobaczyć, czy zmiana tokenizacji na wziętą z NLTK coś zmieni).

In [20]:
import math
import logging
from collections import defaultdict
from itertools import count

In [7]:
WIKI_ARTICLES_FILE_PATH = 'data/fp_wiki.txt'

In [14]:
ARTICLE_IDS_BY_WORD = defaultdict(set)

with open(WIKI_ARTICLES_FILE_PATH) as f:
    lines = iter(f)
    try:
        for i in count():
            _title_with_prefix = next(lines)
            text = next(lines).split()
            while sentence := next(lines).split():
                text.extend(sentence)
            for word in text:
                ARTICLE_IDS_BY_WORD[word.lower()].add(i)
    except StopIteration:
        pass

INDEX = {
    word: sorted(ids)
    for word, ids in ARTICLE_IDS_BY_WORD.items()
}

In [28]:
def get_int32_posting_lists_size_bytes():
    return sum(
        4 * len(posting_list)
        for posting_list in INDEX.values()
    )

In [29]:
def get_vb_posting_lists_size_bytes(bits):
    num_segments = 0
    for posting_list in INDEX.values():
        prev = -1
        for id_ in posting_list:
            diff = id_ - prev
            num_segments += math.ceil(math.log2(diff) / (bits - 1))
    return num_segments * (bits / 8)

In [30]:
def get_gamma_posting_lists_size_bytes():
    total_bytes = 0
    for posting_list in INDEX.values():
        list_bits = 0
        prev = -1
        for id_ in posting_list:
            diff = id_ - prev
            list_bits += 1 + 2 * math.floor(math.log2(diff))
        total_bytes += math.ceil(list_bits / 8)
    return total_bytes

In [31]:
get_int32_posting_lists_size_bytes()

279586592

In [32]:
get_vb_posting_lists_size_bytes(bits=8)

208623045.0

In [33]:
get_vb_posting_lists_size_bytes(bits=4)

235949862.0

In [34]:
get_gamma_posting_lists_size_bytes()

328320429